In [3]:
IS_LOCAL = True
import numpy as np
import pandas as pd
from skimage.io import imread
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
if(IS_LOCAL):
    import pydicom as dicom
else:
    import dicom
import os

In [5]:
if(IS_LOCAL):
    PATH="./input/siim-medical-image/"
else:
    PATH="./input/"
print(os.listdir(PATH))

['dicom_dir.zip', 'full_archive.npz.zip', 'overview.csv', 'tiff_images.zip']


<h1> Read the data</h1>

<h2> Read overview data</h2>

In [6]:
data_df = pd.read_csv(os.path.join(PATH,"overview.csv"))

In [7]:
print("CT Medical images -  rows:",data_df.shape[0]," columns:", data_df.shape[1])

CT Medical images -  rows: 100  columns: 8


In [8]:
data_df.head()

,Unnamed: 0,Age,Contrast,ContrastTag,raw_input_path,id,tiff_name,dicom_name
0,0,60,True,NONE,../data/50_50_dicom_cases\Contrast\00001 (1).dcm,0,ID_0000_AGE_0060_CONTRAST_1_CT.tif,ID_0000_AGE_0060_CONTRAST_1_CT.dcm
1,1,69,True,NONE,../data/50_50_dicom_cases\Contrast\00001 (10).dcm,1,ID_0001_AGE_0069_CONTRAST_1_CT.tif,ID_0001_AGE_0069_CONTRAST_1_CT.dcm
2,2,74,True,APPLIED,../data/50_50_dicom_cases\Contrast\00001 (11).dcm,2,ID_0002_AGE_0074_CONTRAST_1_CT.tif,ID_0002_AGE_0074_CONTRAST_1_CT.dcm
3,3,75,True,NONE,../data/50_50_dicom_cases\Contrast\00001 (12).dcm,3,ID_0003_AGE_0075_CONTRAST_1_CT.tif,ID_0003_AGE_0075_CONTRAST_1_CT.dcm
4,4,56,True,NONE,../data/50_50_dicom_cases\Contrast\00001 (13).dcm,4,ID_0004_AGE_0056_CONTRAST_1_CT.tif,ID_0004_AGE_0056_CONTRAST_1_CT.dcm


<h2> Read TIFF data </h2>

In [10]:
print("Number of TIFF images:", len(os.listdir(os.path.join(PATH,"tiff_images"))))

Number of TIFF images: 100


In [11]:
tiff_data = pd.DataFrame([{'path': filepath} for filepath in glob(PATH+'tiff_images/*.tif')])

<h3> Process TIFF data </h3>
<p>
We define a function to process data.
<br/>
We extract file, ID, age, contrast, modality information from path info.
</p>

In [13]:
def process_data(path):
    data = pd.DataFrame([{'path': filepath} for filepath in glob(PATH+path)])
    data['file'] = data['path'].map(os.path.basename)
    data['ID'] = data['file'].map(lambda x: str(x.split('_')[1]))
    data['Age'] = data['file'].map(lambda x: int(x.split('_')[3]))
    data['Contrast'] = data['file'].map(lambda x: bool(int(x.split('_')[5])))
    data['Modality'] = data['file'].map(lambda x: str(x.split('_')[6].split('.')[-2]))
    return data

In [14]:
tiff_data = process_data('tiff_images/*.tif')

In [15]:
tiff_data.head()

,path,file,ID,Age,Contrast,Modality
0,./input/siim-medical-image/tiff_images\ID_0000...,ID_0000_AGE_0060_CONTRAST_1_CT.tif,0000,60,True,CT
1,./input/siim-medical-image/tiff_images\ID_0001...,ID_0001_AGE_0069_CONTRAST_1_CT.tif,0001,69,True,CT
2,./input/siim-medical-image/tiff_images\ID_0002...,ID_0002_AGE_0074_CONTRAST_1_CT.tif,0002,74,True,CT
3,./input/siim-medical-image/tiff_images\ID_0003...,ID_0003_AGE_0075_CONTRAST_1_CT.tif,0003,75,True,CT
4,./input/siim-medical-image/tiff_images\ID_0004...,ID_0004_AGE_0056_CONTRAST_1_CT.tif,0004,56,True,CT
